In [1]:
import pandas as pd
from thefuzz import fuzz
from helpers import classification as pf
from helpers import stats as bf
from services import BetExplorerService, MySQLService
import os

from helpers.file import load_from_file
from helpers.options import filtered_cols, selected_stats, strategy

C:\Users\caiog\anaconda3\lib\site-packages\thefuzz\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Setup Complete


In [2]:
season = 2023
league = "bundesliga"
country = "germany"

n_last_games = 5

bankroll = 100

In [3]:
path = f"../dist/leagues/{league}/official"

pipeline = pf.load_saved_utils(path)

In [4]:
mysql = MySQLService()

where_clause = f'season = {season}'
order_by_clause = "date ASC"

season_games = mysql.get_data('matches', where_clause=where_clause, order_by_clause=order_by_clause)
season_games["winner"] = season_games.apply(
    lambda x: bf.get_winner(x["home_score"], x["away_score"]), axis=1
)

teams_query = f"SELECT DISTINCT(home_team) as team FROM matches WHERE season = {season}"

teams = mysql.execute_query(teams_query)

In [5]:
display(season_games)

,season,league,date,week,home_team,home_xg,home_score,away_score,away_xg,away_team,...,away_Poss,away_Succ,away_Succ_pct,away_CrdY,away_CrdR,away_Recov,home_odds,away_odds,draw_odds,winner
0,2023,bundesliga,2023-08-18,1,Werder Bremen,0.6,0,4,2.8,Bayern Munich,...,72.0,20.0,48.8,1.0,0.0,61.0,8.56,1.31,5.99,A
1,2023,bundesliga,2023-08-19,1,Augsburg,2.3,4,4,1.7,M'Gladbach,...,44.0,7.0,43.8,2.0,0.0,46.0,2.77,2.42,3.67,D
2,2023,bundesliga,2023-08-19,1,Dortmund,1.8,1,0,1.2,Köln,...,34.0,7.0,53.8,0.0,0.0,48.0,1.45,6.3,4.96,H
3,2023,bundesliga,2023-08-19,1,Hoffenheim,1.4,1,2,2.6,Freiburg,...,50.0,14.0,60.9,1.0,0.0,57.0,2.2,3.11,3.67,A
4,2023,bundesliga,2023-08-19,1,Leverkusen,1.4,3,2,1.6,RB Leipzig,...,56.0,9.0,47.4,2.0,0.0,47.0,2.43,2.79,3.58,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,2023,bundesliga,2024-03-02,24,Mainz 05,1.0,1,1,1.0,M'Gladbach,...,57.0,2.0,20.0,1.0,0.0,43.0,1.92,3.98,3.67,D
212,2023,bundesliga,2024-03-02,24,Union Berlin,1.2,0,2,1.6,Dortmund,...,61.0,9.0,39.1,1.0,0.0,54.0,3.38,2.1,3.65,A
213,2023,bundesliga,2024-03-02,24,Wolfsburg,0.8,2,3,1.1,Stuttgart,...,58.0,7.0,38.9,2.0,0.0,49.0,3.23,2.2,3.55,A
214,2023,bundesliga,2024-03-03,24,Hoffenheim,1.7,2,1,1.3,Werder Bremen,...,53.0,8.0,50.0,1.0,0.0,44.0,1.95,3.51,4.04,H


In [6]:
# Getting odds for next games

bet_explorer = BetExplorerService(country, league)

next_games = bet_explorer.get_next_games()

print(next_games)

[{'home_team': 'FC Koln', 'away_team': 'Darmstadt', 'home_odds': 1.46, 'draw_odds': 4.7, 'away_odds': 6.82, 'date': datetime.datetime(2024, 4, 20, 10, 5, 27, 717681)}, {'home_team': 'Heidenheim', 'away_team': 'RB Leipzig', 'home_odds': 6.13, 'draw_odds': 5.01, 'away_odds': 1.46, 'date': datetime.datetime(2024, 4, 20, 10, 5, 27, 718684)}, {'home_team': 'Hoffenheim', 'away_team': 'B. Monchengladbach', 'home_odds': 2.21, 'draw_odds': 3.89, 'away_odds': 2.97, 'date': datetime.datetime(2024, 4, 20, 10, 5, 27, 718684)}, {'home_team': 'Wolfsburg', 'away_team': 'Bochum', 'home_odds': 1.93, 'draw_odds': 3.65, 'away_odds': 3.95, 'date': datetime.datetime(2024, 4, 20, 10, 5, 27, 718684)}, {'home_team': 'Union Berlin', 'away_team': 'Bayern Munich', 'home_odds': 4.88, 'draw_odds': 4.26, 'away_odds': 1.66, 'date': datetime.datetime(2024, 4, 20, 10, 5, 27, 718684)}, {'home_team': 'Werder Bremen', 'away_team': 'Stuttgart', 'home_odds': 5.01, 'draw_odds': 4.55, 'away_odds': 1.6, 'date': datetime.dateti

In [7]:
def get_most_compatible_team(team_key):
    team_compatibility = teams
    team_compatibility["score"] = team_compatibility.apply(
        lambda x: fuzz.ratio(game[team_key], x["team"]),
        axis=1,
    )
    team_compatibility = team_compatibility.sort_values(
        by="score", ascending=False
    ).reset_index(drop=True)
    
    return team_compatibility.iloc[0]["team"]

In [8]:
data_model = []
for game in next_games:
    home_team_compatible = get_most_compatible_team("home_team")
    game["home_team"] = home_team_compatible
    
    away_team_compatible = get_most_compatible_team("away_team")
    game["away_team"] = away_team_compatible
    
    home_stats_dict = bf.get_team_previous_games_stats(
        game["home_team"], season, game["date"], "H", n_last_games, season_games
    )
    if not home_stats_dict:
        continue

    away_stats_dict = bf.get_team_previous_games_stats(
        game["away_team"], season, game["date"], "A", n_last_games, season_games
    )
    if not away_stats_dict:
        continue

    game_info_keys = [
        "date",
        "season",
        "home_team",
        "away_team",
        "home_odds",
        "away_odds",
        "draw_odds",
        "winner",
        "home_score",
        "away_score",
    ]
    game_info_dict = {key: game.get(key) for key in game_info_keys}

    data_model.append({**home_stats_dict, **away_stats_dict, **game_info_dict})

data_df = pd.DataFrame(data_model)

In [9]:
display(data_df)

,home_points_pct,home_win_pct,home_draw_pct,home_loss_pct,home_points_pct_last_games,home_win_pct_last_games,home_draw_pct_last_games,home_loss_pct_last_games,home_home_wins_last_games,home_home_draws_last_games,...,date,season,home_team,away_team,home_odds,away_odds,draw_odds,winner,home_score,away_score
0,0.236111,0.125000,0.333333,0.541667,0.333333,0.2,0.4,0.4,1,0,...,2024-04-20 10:05:27.717681,None,Köln,Darmstadt 98,1.46,6.82,4.70,None,None,None
1,0.388889,0.291667,0.291667,0.416667,0.333333,0.2,0.4,0.4,0,1,...,2024-04-20 10:05:27.718684,None,Heidenheim,RB Leipzig,6.13,1.46,5.01,None,None,None
2,0.458333,0.375000,0.250000,0.375000,0.533333,0.4,0.4,0.2,1,1,...,2024-04-20 10:05:27.718684,None,Hoffenheim,M'Gladbach,2.21,2.97,3.89,None,None,None
3,0.347222,0.250000,0.291667,0.458333,0.200000,0.0,0.6,0.4,0,2,...,2024-04-20 10:05:27.718684,None,Wolfsburg,Bochum,1.93,3.95,3.65,None,None,None
4,0.347222,0.291667,0.166667,0.541667,0.533333,0.4,0.4,0.2,1,1,...,2024-04-20 10:05:27.718684,None,Union Berlin,Bayern Munich,4.88,1.66,4.26,None,None,None
5,0.416667,0.333333,0.250000,0.416667,0.466667,0.4,0.2,0.4,0,1,...,2024-04-21 10:05:27.718684,None,Werder Bremen,Stuttgart,5.01,1.60,4.55,None,None,None
6,0.611111,0.500000,0.333333,0.166667,0.533333,0.4,0.4,0.2,1,0,...,2024-04-21 10:05:27.718684,None,Dortmund,Leverkusen,2.51,2.60,3.82,None,None,None
7,0.416667,0.333333,0.250000,0.416667,0.133333,0.0,0.4,0.6,0,2,...,2024-04-21 10:05:27.718684,None,Freiburg,Mainz 05,2.63,2.61,3.56,None,None,None
8,0.347222,0.208333,0.416667,0.375000,0.333333,0.2,0.4,0.4,1,1,...,2024-04-26 00:00:00.000000,None,Bochum,Hoffenheim,2.24,2.93,3.79,None,None,None


In [10]:
X, _, odds = pf.separate_dataset_info(data_df)

X = X[filtered_cols]

predictions = pipeline.predict(X)
probabilities = pipeline.predict_proba(X)

probs_test_df = pd.DataFrame(
    probabilities,
    index=data_df.index,
    columns=["away_probs", "draw_probs", "home_probs"],
)
preds_test_df = pd.DataFrame(predictions, index=data_df.index, columns=["pred"])
test_results_df = pd.concat([preds_test_df, probs_test_df, odds], axis=1)

test_results_df.dropna(subset=["home_odds"], inplace=True)
test_results_df = test_results_df[test_results_df["home_odds"] != " "]

test_results_df = test_results_df.astype(
    {"home_odds": float, "draw_odds": float, "away_odds": float}
)

In [11]:
today_bets = 0
for _, game in test_results_df.iterrows():
    bet_value = pf.get_bet_value_by_row(game, bankroll, strategy)
    odds, probs = pf.get_bet_odds_probs(game)
    bet_worth_it = pf.bet_worth_it(bet_value, odds)
    if bet_value < 0:
        continue
    today_bets += 1

    print(f"\n{game['home_team']} ({game['home_odds']})")
    print(f"X ({game['draw_odds']})")
    print(f"{game['away_team']} ({game['away_odds']})")
    print(f"Prediction: {game['pred']} ({odds})")
    print(f"Bet Value: ${round(bet_value, 2)}")
    print(f"{'GOOD' if bet_worth_it else 'BAD'} BET")

if not today_bets:
    print("\nSorry, there are no bets for today.")


Köln (1.46)
X (4.7)
Darmstadt 98 (6.82)
Prediction: A (6.82)
Bet Value: $5.0
GOOD BET

Heidenheim (6.13)
X (5.01)
RB Leipzig (1.46)
Prediction: H (6.13)
Bet Value: $5.0
GOOD BET

Hoffenheim (2.21)
X (3.89)
M'Gladbach (2.97)
Prediction: H (2.21)
Bet Value: $5.0
GOOD BET

Wolfsburg (1.93)
X (3.65)
Bochum (3.95)
Prediction: H (1.93)
Bet Value: $5.0
GOOD BET

Union Berlin (4.88)
X (4.26)
Bayern Munich (1.66)
Prediction: A (1.66)
Bet Value: $5.0
BAD BET

Werder Bremen (5.01)
X (4.55)
Stuttgart (1.6)
Prediction: D (4.55)
Bet Value: $5.0
GOOD BET

Dortmund (2.51)
X (3.82)
Leverkusen (2.6)
Prediction: A (2.6)
Bet Value: $5.0
GOOD BET

Freiburg (2.63)
X (3.56)
Mainz 05 (2.61)
Prediction: H (2.63)
Bet Value: $5.0
GOOD BET

Bochum (2.24)
X (3.79)
Hoffenheim (2.93)
Prediction: H (2.24)
Bet Value: $5.0
GOOD BET
